In [1]:
import numpy as np
import pandas as pd
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# import
# pull QB report Average Days to Pay (filter trans type invoice, ++ name column)
df_trans = pd.read_csv('/kaggle/input/qb-apecco/transactions.CSV',
                       encoding='cp1252')
# pull QB report Job Cost Summary (collapse rows)
df_profit = pd.read_csv('/kaggle/input/qb-apecco/jobprofit.CSV', 
                        encoding='cp1252')

In [3]:
# clean
df_trans.dropna(subset=['Num', 'Avg Days to Pay'], inplace=True)
df_trans['Name'] = df_trans['Name'].str.split(':').str[0]
df_trans['Date'] = pd.to_datetime(df_trans['Date'])
df_profit.rename(columns={'Unnamed: 0': 'Name', '($) Diff.': 'Profit'}, 
                 inplace=True)
df_profit = df_profit[df_profit['Name'] != 'TOTAL']

In [4]:
# prep data
df_merged = pd.merge(df_trans, df_profit, on='Name', how='inner')
df_cust = df_merged.groupby('Name', as_index=False).agg(
    AvgDaysToPay=('Avg Days to Pay', 'mean'),
    TotalProfit=('Profit', 'first'),
    TransCount=('Name', 'count'),
)

df_cust['AvgProfit'] = df_cust['TotalProfit'] / df_cust['TransCount']
df_cust['AvgDaysToPay'] = df_cust['AvgDaysToPay'].clip(lower=0)
df_cust = df_cust[['Name', 'AvgProfit', 'AvgDaysToPay']]

cust_round = {
    'AvgDaysToPay' : 0,
    'AvgProfit' : 0,
}

df_cust = df_cust.round(cust_round)

In [5]:
# scale data, find inertia elbow, get k
numeric_cols = ['AvgProfit', 'AvgDaysToPay']
X = df_cust[numeric_cols].fillna(0)
X_scaled = StandardScaler().fit_transform(X)

inertias = []
k_range = range(1, 15)
for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

k1 = KneeLocator(k_range, inertias, curve='convex', direction='decreasing')
best_k = k1.elbow
print('n clusters: ', best_k)

n clusters:  4


In [6]:
# get k clusters
kmeans = KMeans(n_clusters=best_k, random_state=0, n_init=10)
df_cust['Cluster'] = kmeans.fit_predict(X_scaled)

df_clusters = df_cust.groupby('Cluster')[numeric_cols].mean()
df_clusters['Count'] = df_cust.groupby('Cluster')['Name'].count()
df_clusters['%'] = df_clusters['Count'] / df_clusters['Count'].sum() * 100

cluster_round = {
    'AvgDaysToPay' : 0,
    'AvgProfit' : 0,
    '%': 2,
}

df_clusters = df_clusters.round(cluster_round)

In [7]:
# write into excel
output = 'customer_clusters.xlsx'
with pd.ExcelWriter(output) as writer:
    df_cust.to_excel(writer, sheet_name='custdata', index=False)
    df_clusters.to_excel(writer, sheet_name='clusters')